In [ ]:
import sys
!{sys.executable} -m pip install pymongo azure-storage-blob pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 KB 5.8 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Library init with help from Professor Bien-Aime
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContentSettings,ContainerClient
import os
import gzip
import requests
import pandas as pd
from io import BytesIO
import shutil

In [ ]:
title_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [ ]:
## Code that will download zip file to query the apis with help from Professor Bien-Aime

# Send a HTTP request to the URL
response = requests.get(title_url, stream=True)

gz_file_path = 'title.gz'
# Create a temporary .gz file
with open(gz_file_path, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

df = pd.read_csv(gz_file_path, compression='gzip', header=0, sep='\t')
df.head()

/shared-libs/python3.9/py-core/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [ ]:
## Code that will download zip file to query the apis with help from Professor Bien-Aime

# Send a HTTP request to the URL
response = requests.get(ratings_url, stream=True)

gz_file_path = 'ratings.gz'
# Create a temporary .gz file
with open(gz_file_path, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

rating_df = pd.read_csv(gz_file_path, compression='gzip', header=0, sep='\t')
rating_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1986
1,tt0000002,5.8,265
2,tt0000003,6.5,1845
3,tt0000004,5.5,178
4,tt0000005,6.2,2627


In [ ]:
merged_df = pd.merge(df, rating_df, on='tconst', how='inner')
merged_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,1986
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",5.8,265
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1845
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5.5,178
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2627


In [ ]:
merged_df.shape

(1328584, 11)

In [ ]:
# setup azure blob connection
AZURE_STORAGE_CONNECTION_STRING ='DefaultEndpointsProtocol=https;AccountName=stboxoffice;AccountKey=8PVYmm5+zCALb3D68QpEAa7Ceuy/79j8pQInN0Go7fYgOHsIVebBnbUuItZvwJyf8TjH5WIRa9ss+AStmE8kew==;EndpointSuffix=core.windows.net'
container_name = 'stboxoffice'
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(container_name)


csv_data = merged_df.to_csv(index=False)

# define the blob name
blob_name = 'processed/imdb_merged_20230712.csv'

# get a blob client
blob_client = blob_service_client.get_blob_client(container_name, blob_name)

# upload the CSV data to Azure Blob Storage
blob_client.upload_blob(csv_data)

{'etag': '"0x8DB82A0329DD191"',
 'last_modified': datetime.datetime(2023, 7, 12, 6, 21, 15, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'\xde\x18\xa4\x14\xc0d\xfe]'),
 'client_request_id': '4e6aa592-207c-11ee-9009-ba327c695ab0',
 'request_id': '5977d2f5-f01e-004b-7489-b4757f000000',
 'version': '2023-01-03',
 'version_id': None,
 'date': datetime.datetime(2023, 7, 12, 6, 21, 15, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

In [ ]:
# setup azure blob connection
AZURE_STORAGE_CONNECTION_STRING ='DefaultEndpointsProtocol=https;AccountName=stboxoffice;AccountKey=8PVYmm5+zCALb3D68QpEAa7Ceuy/79j8pQInN0Go7fYgOHsIVebBnbUuItZvwJyf8TjH5WIRa9ss+AStmE8kew==;EndpointSuffix=core.windows.net'
container_name = 'stboxoffice'
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(container_name)


# select unique value of a column name title
unique_titles = df['originalTitle'].unique().tolist()

unique_tconst = df['tconst'].unique().tolist()

print(len(unique_tconst))

# initialize an empty list to hold API responses
omdb_responses = []
count = 0
# loop through that list call api
for tconst in unique_tconst:
    # assuming you are calling an API that takes title as a parameter
    # replace with actual API endpoint
    # http://www.omdbapi.com/?i=tt3896198&apikey=478896bc
    response = requests.get(f"http://www.omdbapi.com/?i={tconst}&apikey=478896bc") 
    # append the response to the list
    count = count + 1 
    print(count)
    omdb_responses.append(response.json())
    
# create a dataframe from the API responses
omdb_df = pd.DataFrame(omdb_responses)
omdb_df.head()

# convert the dataframe to a CSV string
csv_data = omdb_df.to_csv(index=False)
omdb_df.to_csv("api_responses.csv")

# define the blob name
blob_name = 'processed/api_responses_20230712.csv'

# get a blob client
blob_client = blob_service_client.get_blob_client(container_name, blob_name)

# upload the CSV data to Azure Blob Storage
blob_client.upload_blob(csv_data)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5a41f01c-4a5e-4491-9540-135942f8c6b7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>